#### Educational Facilities Scrape - Wrangling

In [1]:
import numpy as np 
import pandas as pd
from postal.parser import parse_address

from geopy import *
import geopandas
from geopy.extra.rate_limiter import RateLimiter

import re
import time
import glob

In [2]:
locator = Nominatim(user_agent="myGeocoder", country_bias = "CAN")

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `country_bias` argument of the Nominatim.__init__ is deprecated and will be removed in geopy 2.0. Use Nominatim.geocode(country_codes='CAN') instead.
  """Entry point for launching an IPython kernel.


In [3]:
# path = '../Collection-ODECF/data/education/'
# file = 'AB-education.csv'
# df = pd.read_csv(path + file)

In [4]:
paths = glob.glob('../Collection-ODECF/data/education/*-education.csv')
files = ['QU', 'NL', 'YU', 'MB', 'NT', 'NB', 'NS', 'AB', 'PE', 'BC', 'SK', 'ON']

In [10]:
for p,f in zip(paths, files):
    df = pd.read_csv(p)
    
    # clean address column
    print("{} - cleaning address column".format(f))
    phone = df.Address.str.extract(r'([(]{0,1}[0-9]{3}[)]{0,1}\s{0,1}\-{0,1}[0-9]{3}\-{0,1}[0-9]*)')
    df['phone'] = phone
    df.Address = df.Address.str.replace(r'([(]{0,1}[0-9]{3}[)]{0,1}\s{0,1}\-{0,1}[0-9]{3}\-{0,1}[0-9]*)', '')
    
    # add missing columns
    print("{} - adding provider and province columns".format(f))
    df['provider'] = 'ESDC'
    df['province'] = f.lower()
    
    # geocode
#     t = time.localtime(time.time())
#     print("Geocoding {}".format(f))
#     print("Time of epoch = {}:{}:{}".format(t.tm_hour,t.tm_min, t.tm_sec))

#         # 1 - convenient function to delay between geocoding calls
#     geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

#         # 2 - create location column
#     temp_loc = df.Address.apply(geocode)
    
#         # 3 - create latitude and longitude cols from temp_loc (returns tuple)
#     df['latitude'] = temp_loc.apply(lambda loc: tuple(loc.point)[0] if loc else None)
#     df['longitude'] = temp_loc.apply(lambda loc: tuple(loc.point)[1] if loc else None)

#     print("percent coverage: {}".format(sum(df.latitude.notnull())/len(df)*100))
    
    # export
    print("{} - export".format(f.lower()))
    df.to_csv("output/education/{}-education.csv".format(f.lower()))

QU - cleaning address column
QU - adding provider and province columns
qu - export
NL - cleaning address column
NL - adding provider and province columns
nl - export
YU - cleaning address column
YU - adding provider and province columns
yu - export
MB - cleaning address column
MB - adding provider and province columns
mb - export
NT - cleaning address column
NT - adding provider and province columns
nt - export
NB - cleaning address column
NB - adding provider and province columns
nb - export
NS - cleaning address column
NS - adding provider and province columns
ns - export
AB - cleaning address column
AB - adding provider and province columns
ab - export
PE - cleaning address column
PE - adding provider and province columns
pe - export
BC - cleaning address column
BC - adding provider and province columns
bc - export
SK - cleaning address column
SK - adding provider and province columns
sk - export
ON - cleaning address column
ON - adding provider and province columns
on - export


In [12]:
df

,Unnamed: 0,EI Code,EI Name,EI Type,Address,EI Designation Date,phone,provider,province
0,0,EJAE,Stanford International College of Business & T...,Junior College,"25 Scarsdale Rd., Units 8 & 9, Toronto, Ont...",2013-11-04,4164238188,ESDC,on
1,1,EJAF,The Audio Recording Academy,Junior College,"1540 Raven Avenue, Ottawa, Ontario, Canada,",2017-01-11,6137980070,ESDC,on
2,2,EPAE,Canadian Business College - Mississauga,Private Institution,"77 City Centre Drive, Suite 600, Mississauga...",2006-06-09,905-279-9929,ESDC,on
3,3,EPAF,Canadian Mothercraft Society,Private Institution,"646 St Clair Avenue West, Toronto, Ontario, ...",1988-12-21,(416) 483-0511,ESDC,on
4,4,EPAL,Marca College of Hair and Esthetics-Mississauga,Private Institution,"4141 Dixie Road, Mississauga, Ontario, Cana...",2008-01-10,(416) 698-2558,ESDC,on
...,...,...,...,...,...,...,...,...,...
272,272,EVCN,College D'Alfred,Technical and Vocational,"31 Rue St. Paul, Alfred, Ontario, Canada,",1988-12-21,6136792218,ESDC,on
273,273,EVDB,La Cite Collegiale,Technical and Vocational,"801 Aviation Parkway, Ottawa, Ontario, Cana...",1990-04-25,6137862400,ESDC,on
274,274,EVDQ,College Boreal,Technical and Vocational,"21 Boulevard Lasalle, Sudbury, Ontario, Can...",1995-03-15,7055606673,ESDC,on
275,275,EVDU,Algonquin College of Applied Arts & Technology...,Technical and Vocational,"1 College Way, Pembroke, Ontario, Canada,",1998-01-19,6137354700,ESDC,on


---
Parse Address

In [ ]:
res1 = [('6730', 'house_number'), ('university drive', 'road'), ('lacombe', 'city'), ('alberta', 'state'), ('canada', 'country')]
res2 = [('university drive', 'road'), ('lacombe', 'city'), ('alberta', 'state'), ('canada', 'country')]

def ishousenum(res):
    """ does the house_number already exist in the tuple?"""
    for tup in res:
        if 'house_number' == tup[1]:
            return True

In [ ]:
def parse_all(x):
    res = parse_address(x)
    
    if ishousenum(res) == True:
        pass
    else:
        res.append(('none', 'house_number'))
        
    return res

parsed = df.Address.map(parse_all)

In [ ]:
parsed

In [ ]:
def Convert(tup, di):
    for v, k in tup:
        di.setdefault(k, []).append(v) 
    return di 
      
dictionary = {} 
parsed.map(lambda x: Convert(x, dictionary))

In [ ]:
len(dictionary['postcode'])

In [ ]:
dictionary

In [ ]:
df['street_number'] = dictionary['house_number']
df['street_name'] = dictionary['road']
df['city'] = dictionary['city']
df['unit'] = dictionary['unit']
df['postal_code'] = dictionary['postcode']

In [ ]:
ab['street_name'] = str_name
ab['street_number'] = str_num